In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


In [6]:
prompt = ChatPromptTemplate.from_template("rell me a curios thing about {Football_player}")

output_parser = StrOutputParser()

In [10]:
chain = prompt | chatModel | output_parser

chain.invoke({"Football_player": "Ronaldo"})


"One curious thing about Cristiano Ronaldo is that he owns a hotel in his hometown of Funchal, Madeira, called the Pestana CR7 Hotel. The hotel is a luxury boutique hotel that reflects Ronaldo's style and persona, featuring his personal brand throughout the property."

# Use of .bind() to add arguments to a Runnable in a LCEL Chain

For example, we can add an argument to stop the model response when it reaches the word "Ronaldo":

In [15]:
chain = prompt | chatModel.bind(stop=["Ronaldo"]) | output_parser


# Combining LCEL chains

In [16]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")

chain = prompt | model | StrOutputParser()


In [19]:
chain.invoke("İsmetinönü")

'İsmet İnönü was a prominent Turkish politician and the second President of Turkey.'

# Conbined chain

In [20]:
historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for Humanity?")

composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()


In [22]:
composed_chain.invoke("Mustafa Kemal Atatürk")

"Yes, Mustafa Kemal Atatürk was a visionary leader who played a crucial role in modernizing Turkey and advocating for secularism and cultural reform. His reforms had a significant impact on shaping Turkey into a more modern and progressive society, and his emphasis on secularism helped to create a more inclusive and tolerant society. Overall, Atatürk's contributions have been seen as positive for humanity, as they have helped to promote progress, modernization, and secular values in Turkey and beyond."

# Another Example: a chain inside another chain

In [24]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("What is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "What continent is the country {country} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"politician": "Atatürk", "language":"Turkish"})

"Mustafa Kemal Atatürk, Türkiye'den gelmektedir. Türkiye, Asya ve Avrupa kıtaları arasında bulunan bir ülkedir."

# LCEL chain at work in typical RAG app

In [25]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        ),
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [26]:
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is a technique used to break down complex tasks into smaller and simpler steps for easier execution. Different approaches like using simple prompting, task-specific instructions, or external planners can be employed for task decomposition. It aims to make tasks more manageable and improve model performance on complex tasks.'